In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pylhef
import sys
%matplotlib notebook



In [ ]:
lhfile = pylhef.read('test_events.lhe')

sim_events = []

for event in lhfile.events:
    part = event.particles[2]
    pid = part.id
    inout = 1
    fmom = part.first_mother
    lmom = part.last_mother
    col1 = part.color[0]
    col2 = part.color[1]
    px = part.p[1]
    py = part.p[2]
    pz = part.p[3]
    energy = part.p[0]
    mas = part.mass
    lt = part.lifetime
    spi = part.spin
    
    simpart = [pid,inout,fmom,lmom,col1,col2,px,py,pz,energy,mas,lt,spi]
    sim_events.append(simpart)

In [ ]:
test=open('test_events.lhe','r')
file=[]

output = ""
for line in test:
    file.append(line)

# This is where each event starts
lines=[i for i,x in enumerate(file) if x=='<event>\n']
len(lines) # = 94

sims=[]
n=0
while n<=len(lines):
    if n==0:
        place = lines[n]+5
        sims.append(file[0:place])
        sims.append(sim_events[n])
    elif n < len(lines):
        place = lines[n]+5
        last = lines[n-1]+5
        sims.append(file[last:place])
        sims.append(sim_events[n])
    else:
        last = lines[n-1]+5
        sims.append(file[last:])
    n=n+1
    
head = sims[0][0:-5]
foot = sims[-1][-1]

In [ ]:
foot

In [ ]:
h = ('').join(head)
print(foot)

In [ ]:
len(sims[2])

Myrl, can you figure out what all this stuff is and make notes of it here. 

In [ ]:
<event>
 4      1 +5.0558200e+02 2.08121300e+02 7.54677100e-03 1.14306200e-01
       21 -1    0    0  503  502 +0.0000000000e+00 +0.0000000000e+00 +4.6699672078e+02 4.6699672078e+02 0.0000000000e+00 0.0000e+00 -1.0000e+00
       21 -1    0    0  501  503 -0.0000000000e+00 -0.0000000000e+00 -1.7306846547e+02 1.7306846547e+02 0.0000000000e+00 0.0000e+00 -1.0000e+00
        6  1    1    2  501    0 -1.0549343684e+02 +4.7503664050e+01 +3.6498091942e+02 4.2014941967e+02 1.7300000000e+02 0.0000e+00 -1.0000e+00
       -6  1    1    2    0  502 +1.0549343684e+02 -4.7503664050e+01 -7.1052664100e+01 2.1991576658e+02 1.7300000000e+02 0.0000e+00 -1.0000e+00
<mgrwt>
<rscale>  2 0.20812127E+03</rscale>
<asrwt>0</asrwt>
<pdfrwt beam="1">  1       21 0.71845649E-01 0.20812127E+03</pdfrwt>
<pdfrwt beam="2">  1       21 0.26625918E-01 0.20812127E+03</pdfrwt>
<totfact> 0.23273598E+04</totfact>
</mgrwt>
</event>

In [ ]:
# Print out header to file
#
# Loop over LHE events/read and do modification
## Make a string to write out 
#
# Print out foot